In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
data_train = pd.read_csv("./data/genresTrain.csv")
data_train.head()

,PAR_TC,PAR_SC,PAR_SC_V,PAR_ASE1,PAR_ASE2,PAR_ASE3,PAR_ASE4,PAR_ASE5,PAR_ASE6,PAR_ASE7,...,PAR_3RMS_TCD,PAR_ZCD_10FR_MEAN,PAR_ZCD_10FR_VAR,PAR_1RMS_TCD_10FR_MEAN,PAR_1RMS_TCD_10FR_VAR,PAR_2RMS_TCD_10FR_MEAN,PAR_2RMS_TCD_10FR_VAR,PAR_3RMS_TCD_10FR_MEAN,PAR_3RMS_TCD_10FR_VAR,GENRE
0,2.5788,481.45,76989.0,-0.12334,-0.11578,-0.11176,-0.10412,-0.106100,-0.11026,-0.11375,...,0.002032,0.041056,0.000091,0.049627,0.000231,0.011265,0.000024,0.002177,0.000003,Pop
1,2.7195,1405.30,825380.0,-0.17655,-0.18323,-0.17773,-0.17057,-0.166440,-0.16174,-0.15371,...,0.005959,0.082830,0.001122,0.093622,0.003869,0.031467,0.000779,0.006716,0.000041,Blues
2,2.5351,601.09,686240.0,-0.13940,-0.13251,-0.11486,-0.10173,-0.099342,-0.10936,-0.12668,...,0.002766,0.043652,0.000358,0.045415,0.000211,0.013850,0.000038,0.003069,0.000002,Pop
3,2.4465,637.73,122580.0,-0.14995,-0.14802,-0.13800,-0.12927,-0.125150,-0.12340,-0.12159,...,0.002503,0.045159,0.000356,0.037388,0.000118,0.011897,0.000045,0.002645,0.000006,Jazz
4,2.5657,776.86,124010.0,-0.16863,-0.16112,-0.15935,-0.15120,-0.140340,-0.13002,-0.12804,...,0.002703,0.047347,0.000302,0.049469,0.000606,0.014052,0.000152,0.003244,0.000029,Jazz


In [3]:
data_train.loc[data_train.loc[:,"GENRE"]=="Blues", "GENRE"] = 1
data_train.loc[data_train.loc[:,"GENRE"]=="Classical", "GENRE"] = 2
data_train.loc[data_train.loc[:,"GENRE"]=="Jazz", "GENRE"] = 3
data_train.loc[data_train.loc[:,"GENRE"]=="Metal", "GENRE"] = 4
data_train.loc[data_train.loc[:,"GENRE"]=="Pop", "GENRE"] = 5
data_train.loc[data_train.loc[:,"GENRE"]=="Rock", "GENRE"] = 6

In [4]:
data_test = pd.read_csv("./data/genresTest.csv")
data_test.head()

,PAR_TC,PAR_SC,PAR_SC_V,PAR_ASE1,PAR_ASE2,PAR_ASE3,PAR_ASE4,PAR_ASE5,PAR_ASE6,PAR_ASE7,...,PAR_2RMS_TCD,PAR_3RMS_TCD,PAR_ZCD_10FR_MEAN,PAR_ZCD_10FR_VAR,PAR_1RMS_TCD_10FR_MEAN,PAR_1RMS_TCD_10FR_VAR,PAR_2RMS_TCD_10FR_MEAN,PAR_2RMS_TCD_10FR_VAR,PAR_3RMS_TCD_10FR_MEAN,PAR_3RMS_TCD_10FR_VAR
0,2.5851,887.72,203130.0,-0.17260,-0.16509,-0.15114,-0.14272,-0.13747,-0.13437,-0.12990,...,0.014458,0.003283,0.060789,0.000173,0.049469,0.000209,0.014727,0.000058,0.003690,0.000007
1,2.4621,370.86,4835.0,-0.16553,-0.16676,-0.16022,-0.15304,-0.14354,-0.13443,-0.12450,...,0.007165,0.001832,0.019472,0.000006,0.021197,0.000100,0.008136,0.000055,0.002112,0.000006
2,2.5894,536.33,59175.0,-0.14433,-0.15838,-0.15151,-0.13966,-0.12591,-0.11795,-0.11744,...,0.009742,0.002032,0.037165,0.000158,0.037856,0.000123,0.009627,0.000068,0.001916,0.000004
3,2.4876,1053.70,147250.0,-0.12813,-0.11979,-0.12223,-0.11881,-0.12020,-0.12702,-0.13608,...,0.020571,0.002458,0.074786,0.000113,0.088109,0.000152,0.021083,0.000071,0.002590,0.000006
4,2.7968,354.90,7610.9,-0.16335,-0.16976,-0.17196,-0.16772,-0.16637,-0.15903,-0.13995,...,0.004825,0.001905,0.017377,0.000015,0.010852,0.000153,0.004702,0.000044,0.002101,0.000008


In [5]:
m_cor = data_train.corr()

dict_l = {}
for i in m_cor.index:
    dict_l.update({i:m_cor[m_cor[i]>0.9].index})
    
cols_drop = []
for idx in dict_l:
    for j in range(len(dict_l[idx])):
        if idx != dict_l[idx][j]: #si tiene mas de un valor, el mismo no cuenta
            cols_drop.append(dict_l[idx][j])
            #print(idx, dict_l[idx][j])

In [6]:
len(np.unique(cols_drop))

74

In [7]:
df_train_drop = data_train
df_test_drop = data_test
for idx in np.unique(cols_drop):
    df_train_drop = df_train_drop.drop(idx,axis=1)
    df_test_drop = df_test_drop.drop(idx,axis=1)

In [8]:
df_train_drop.shape, df_test_drop.shape

((12495, 118), (5225, 117))

In [9]:
for i in df_train_drop.columns:
    if sum(df_train_drop[i] > 100) > (df_train_drop.shape[0]/2):
        df_train_drop = df_train_drop.drop(i, axis=1)
        df_test_drop = df_test_drop.drop(i, axis=1)

In [39]:
y_trdrop = df_train_drop["GENRE"]
X_trdrop = df_train_drop.drop("GENRE",axis=1)

clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.2)
clf.fit(X_trdrop, y_trdrop)
print(clf.score(X_trdrop, y_trdrop))

predict = clf.predict(df_test_drop)
result = pd.DataFrame(data=predict, index=np.arange(1, data_test.shape[0]+1), columns=['Genres'])
result.index.name = "Id"
result.to_csv('./results/submission_test_29.csv')

0.9965586234493797


#### Escalar carateristicas

In [10]:
dict_col = {}
for col in X_trdrop.columns:
    dict_col.update({col:[min(X_trdrop[col]), max(X_trdrop[col]), np.mean(X_trdrop[col]), np.median(X_trdrop[col])]})  

NameError: name 'X_trdrop' is not defined

In [48]:
del d_a

In [50]:
dict_col

{'PAR_TC': [0.8377, 4.4046, 2.4892254629852073, 2.499],
 'PAR_ASE8': [-0.22003000000000003,
  -0.06982000000000001,
  -0.12710152509003608,
  -0.12595],
 'PAR_ASE9': [-0.20936, -0.064676, -0.1279121115646253, -0.12739],
 'PAR_ASE10': [-0.20371, -0.059563, -0.12788326266506547, -0.12811],
 'PAR_ASE11': [-0.20626, -0.053818, -0.12820067795117995, -0.12794],
 'PAR_ASE12': [-0.19441,
  -0.057411000000000004,
  -0.12924683393357317,
  -0.12899000000000002],
 'PAR_ASE13': [-0.19822, -0.066224, -0.13233787659063648, -0.13263],
 'PAR_ASE14': [-0.20073, -0.060466, -0.13305252549019564, -0.13399],
 'PAR_ASE15': [-0.19834000000000002, -0.06275, -0.13675391932773184, -0.13715],
 'PAR_ASE16': [-0.21029, -0.074632, -0.13989451764705932, -0.13978],
 'PAR_ASE17': [-0.20263, -0.063789, -0.14327168235294105, -0.14288],
 'PAR_ASE18': [-0.20457999999999998,
  -0.076366,
  -0.14673538031212505,
  -0.14704],
 'PAR_ASE19': [-0.20377, -0.071332, -0.1488617860744298, -0.14877],
 'PAR_ASE20': [-0.20584000000000

### Se revisará columna por columna para ver cuales se puedes escalar

In [54]:
dict_col.keys()

dict_keys(['PAR_TC', 'PAR_ASE8', 'PAR_ASE9', 'PAR_ASE10', 'PAR_ASE11', 'PAR_ASE12', 'PAR_ASE13', 'PAR_ASE14', 'PAR_ASE15', 'PAR_ASE16', 'PAR_ASE17', 'PAR_ASE18', 'PAR_ASE19', 'PAR_ASE20', 'PAR_ASE21', 'PAR_ASE22', 'PAR_ASE23', 'PAR_ASE24', 'PAR_ASE25', 'PAR_ASE26', 'PAR_ASE27', 'PAR_ASE28', 'PAR_ASE32', 'PAR_ASE33', 'PAR_ASE34', 'PAR_ASE_M', 'PAR_ASEV7', 'PAR_ASEV8', 'PAR_ASEV9', 'PAR_ASEV10', 'PAR_ASEV11', 'PAR_ASEV12', 'PAR_ASEV13', 'PAR_ASEV14', 'PAR_ASEV15', 'PAR_ASEV16', 'PAR_ASEV17', 'PAR_ASEV18', 'PAR_ASEV19', 'PAR_ASEV20', 'PAR_ASEV21', 'PAR_ASEV22', 'PAR_ASEV23', 'PAR_ASEV24', 'PAR_ASEV25', 'PAR_ASEV26', 'PAR_ASEV27', 'PAR_ASEV28', 'PAR_ASEV29', 'PAR_ASEV30', 'PAR_ASEV31', 'PAR_ASEV32', 'PAR_ASEV33', 'PAR_ASEV34', 'PAR_ASE_MV', 'PAR_ASC', 'PAR_ASC_V', 'PAR_ASS', 'PAR_ASS_V', 'PAR_SFM1', 'PAR_SFM2', 'PAR_SFM3', 'PAR_SFM4', 'PAR_SFM5', 'PAR_SFM6', 'PAR_SFM7', 'PAR_SFM8', 'PAR_SFM9', 'PAR_SFM10', 'PAR_SFM17', 'PAR_SFM21', 'PAR_SFM22', 'PAR_SFM23', 'PAR_SFM24', 'PAR_SFM_M', 'PAR_S

In [11]:
dict_col["PAR_3RMS_TCD_10FR_VAR"]

KeyError: 'PAR_3RMS_TCD_10FR_VAR'

In [12]:
col_scalar = ["PAR_TC", "PAR_ASC", "PAR_ASS", "PAR_PEAK_RMS_TOT", "PAR_PEAK_RMS10FR_MEAN"]
col_outliers = ["PAR_ASC_V", "PAR_PEAK_RMS10FR_VAR"]

### Primero se probará solo escalando las columas  y manteniendo las columnas con outliers

In [18]:
def column_scale(df, l_cols):
    """
    df: DataFrame de entrada
    l_cols: lista de columnas a escalar
    return: retorna un df con las columnas estandarizadas así:
    (X - mean(X))/(max(X)-min(X))
    """
    for col in l_cols:
        #se cambia cada columna
        df[col] = (df[col] - np.mean(df[col]))/(max(df[col]) - min(df[col]))
    return df

In [23]:
X_trdrop = column_scale(X_trdrop, col_scalar)
X_testdrop = df_test_drop
X_testdrop = column_scale(X_testdrop, col_scalar)

In [179]:
y_trdrop = df_train_drop["GENRE"]

clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.2)
clf.fit(X_trdrop, y_trdrop)
print(clf.score(X_trdrop, y_trdrop))

predict = clf.predict(X_testdrop)
result = pd.DataFrame(data=predict, index=np.arange(1, data_test.shape[0]+1), columns=['Genres'])
result.index.name = "Id"
result.to_csv('./results/submission_test_30.csv')

0.99671868747499


### Eliminando los outliers

In [26]:
X_trdrop = df_train_drop.drop("GENRE",axis=1)
X_trdrop = X_trdrop.drop(col_outliers, axis=1)

X_testdrop = df_test_drop.drop(col_outliers, axis=1)

y_trdrop = df_train_drop["GENRE"]

clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.2)
clf.fit(X_trdrop, y_trdrop)
print(clf.score(X_trdrop, y_trdrop))

predict = clf.predict(X_testdrop)
result = pd.DataFrame(data=predict, index=np.arange(1, data_test.shape[0]+1), columns=['Genres'])
result.index.name = "Id"
result.to_csv('./results/submission_test_31.csv')

0.9971988795518207


In [25]:
X_trdrop.head()

,PAR_TC,PAR_ASE8,PAR_ASE9,PAR_ASE10,PAR_ASE11,PAR_ASE12,PAR_ASE13,PAR_ASE14,PAR_ASE15,PAR_ASE16,...,PAR_THR_2RMS_10FR_MEAN,PAR_THR_2RMS_10FR_VAR,PAR_THR_3RMS_10FR_MEAN,PAR_THR_3RMS_10FR_VAR,PAR_PEAK_RMS_TOT,PAR_PEAK_RMS10FR_MEAN,PAR_ZCD_10FR_VAR,PAR_1RMS_TCD_10FR_VAR,PAR_2RMS_TCD_10FR_VAR,PAR_3RMS_TCD_10FR_VAR
0,0.025113,-0.12465,-0.13659,-0.13951,-0.14003,-0.13893,-0.14148,-0.14381,-0.13480,-0.14171,...,0.021437,0.000036,0.002384,4.177900e-06,-0.048844,-0.002902,0.000091,0.000231,0.000024,0.000003
1,0.064559,-0.14458,-0.14691,-0.15324,-0.14565,-0.14155,-0.15822,-0.11068,-0.11531,-0.11116,...,0.024800,0.000068,0.002770,5.629600e-06,-0.011076,-0.001723,0.001122,0.003869,0.000779,0.000041
2,0.012861,-0.12948,-0.12599,-0.12750,-0.14031,-0.13485,-0.12867,-0.14073,-0.15545,-0.15016,...,0.028207,0.000065,0.002868,5.601000e-06,-0.052747,-0.003049,0.000358,0.000211,0.000038,0.000002
3,-0.011978,-0.12010,-0.12907,-0.12677,-0.12070,-0.13761,-0.13426,-0.12630,-0.12385,-0.14257,...,0.030237,0.000089,0.004931,2.591300e-05,-0.008712,-0.001919,0.000356,0.000118,0.000045,0.000006
4,0.021440,-0.13167,-0.13584,-0.13075,-0.12450,-0.11369,-0.11557,-0.10882,-0.11627,-0.13813,...,0.020022,0.000032,0.000669,9.754300e-07,0.013285,-0.001176,0.000302,0.000606,0.000152,0.000029


### Probar solo eliminando los outliers, tal parece que aumenta el rendimiento, despues, probar esos mismos datos con varios modelos, hay mucho overfitting!!!!

In [13]:
y_trdrop = df_train_drop["GENRE"]
X_trdrop = df_train_drop.drop("GENRE",axis=1)

X_trdrop = X_trdrop.drop(col_outliers, axis=1)
X_testdrop = df_test_drop.drop(col_outliers, axis=1)

clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.2)
clf.fit(X_trdrop, y_trdrop)
print(clf.score(X_trdrop, y_trdrop))

predict = clf.predict(X_testdrop)
result = pd.DataFrame(data=predict, index=np.arange(1, data_test.shape[0]+1), columns=['Genres'])
result.index.name = "Id"
result.to_csv('./results/submission_test_32.csv')

0.9971988795518207


In [12]:
def class_bal(df, label):
    """
    Dado un dataframe este devuelve otro DataFreame el cual contiene todas las clases equilibradas,
    solo elige de forma aleatoria los datos para que sean iguales.
    label: columna target
    """
    min_label = 0
    #Calcular el menor número de elementos de una clase
    for idx in df[label].unique():
        if sum(df[label] == idx) > min_label:
            min_label = sum(df[label] == idx)
        elif sum(df[label] == idx) < min_label:
            min_label = sum(df[label] == idx)
    #creación del dataframe de salida
    df_new = pd.DataFrame(columns=df.columns)
    for i in df[label].unique():
        aux_df = df.loc[df[label] == i].sample(min_label)
        df_new = pd.concat([df_new, aux_df])
        
    return df_new

#### Prueba con las clases balanceadas

In [13]:
X_trdrop = class_bal(df_train_drop, "GENRE")
y_trdrop = X_trdrop["GENRE"]
X_trdrop = X_trdrop.drop("GENRE",axis=1)

X_trdrop = X_trdrop.drop(col_outliers, axis=1)
X_testdrop = df_test_drop.drop(col_outliers, axis=1)

In [21]:
X_trdrop.shape, y_trdrop.shape

((5544, 114), (5544,))

In [22]:
clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.2)
clf.fit(X_trdrop,y_trdrop.astype('int'))
print(clf.score(X_trdrop, y_trdrop.astype('int')))

predict = clf.predict(X_testdrop)
result = pd.DataFrame(data=predict, index=np.arange(1, data_test.shape[0]+1), columns=['Genres'])
result.index.name = "Id"
result.to_csv('./results/submission_test_33.csv')

1.0


In [34]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=0.0001,random_state=1, activation="tanh",hidden_layer_sizes=200)
clf.fit(X_trdrop, y_trdrop.astype('int'))
print(clf.score(X_trdrop, y_trdrop.astype('int')))

predict = clf.predict(X_testdrop)
result = pd.DataFrame(data=predict, index=np.arange(1, data_test.shape[0]+1), columns=['Genres'])
result.index.name = "Id"
result.to_csv('./results/submission_test_35.csv')

0.8383353341336535


In [35]:
X_trdrop.shape

(12495, 114)